In [142]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn import tree

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

Link to Assignment:
https://docs.google.com/document/d/1savsuz2g5zOVs9igef1dF5xuueomoS8Eow8Qr9rMifg/edit

# Task 1
Load the dataset into a Python notebook and test the performance of the following models:
a k-Nearest Neighbour
b Decision Tree
c Logistic Regression
d Random Forest


In [143]:
df = pd.read_csv('Dry_Bean_Dataset.csv')
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097,759.696,288.721612,185.944705,1.552728,0.765002,42508,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
13607,42101,757.499,281.576392,190.713136,1.476439,0.735702,42494,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
13608,42139,759.321,281.539928,191.187979,1.472582,0.734065,42569,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
13609,42147,763.779,283.382636,190.275731,1.489326,0.741055,42667,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [144]:
X = df.drop(columns='Class')
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### a. k-Nearest Neighbour

In [145]:
kNN = KNeighborsClassifier(n_neighbors=7, weights='uniform', metric='cosine')
kNN.fit(X_train, y_train)
y_pred = kNN.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

conf_matrix = confusion_matrix(y_test, y_pred)

print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.8921539817807816
Precision: 0.8921539817807816
Recall: 0.8921539817807816

Confusion Matrix:
[[260   0  64   0   4   0  16]
 [  0 142   0   0   0   0   0]
 [ 16   0 371   0  11   1   3]
 [  0   0   0 785   1  12  53]
 [  6   0  11   8 450   0  11]
 [  6   0   0  19   2 463  23]
 [  3   0   2  81  11   3 565]]


### b. Decision Tree

In [146]:
dT = DecisionTreeClassifier(min_impurity_decrease=0.0003)
dT.fit(X_train, y_train)
y_pred = dT.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

conf_matrix = confusion_matrix(y_test, y_pred)

print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.9150749338818689
Precision: 0.9150749338818689
Recall: 0.9150749338818689

Confusion Matrix:
[[301   0  28   0   2   2  11]
 [  0 142   0   0   0   0   0]
 [ 20   0 373   0   6   2   1]
 [  0   0   0 808   2   7  34]
 [  4   0   7   4 464   0   7]
 [  3   0   0  31   0 467  12]
 [  3   0   3  82   9   9 559]]


### c. Logistic regression

In [149]:
lR = LogisticRegression(max_iter=30000)
lR.fit(X_train, y_train)
y_pred = lR.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

conf_matrix = confusion_matrix(y_test, y_pred)

print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.9133117837202468
Precision: 0.9133117837202468
Recall: 0.9133117837202468

Confusion Matrix:
[[305   0  26   0   1   3   9]
 [  0 142   0   0   0   0   0]
 [ 12   0 379   0   6   2   3]
 [  0   0   0 759   0  19  73]
 [  3   0   6   6 463   0   8]
 [  8   0   0  16   0 475  14]
 [  1   0   2  57   9  11 585]]


### d. Randomforest

In [148]:
rFC = RandomForestClassifier(n_estimators=250, criterion='gini')
rFC.fit(X_train, y_train)
y_pred = rFC.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

conf_matrix = confusion_matrix(y_test, y_pred)

print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.9188950925653835
Precision: 0.9188950925653835
Recall: 0.9188950925653835

Confusion Matrix:
[[307   0  23   0   1   4   9]
 [  0 142   0   0   0   0   0]
 [ 17   0 376   0   5   2   2]
 [  0   0   0 780   2  12  57]
 [  2   0   6   3 468   0   7]
 [  3   0   0  18   0 480  12]
 [  0   0   2  69   8  12 574]]
